In [39]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import SpectralClustering
from sklearn.decomposition import PCA
from sklearn import preprocessing
import copy
import os

### 
### This includes code copied and pasted from the main methods used for the website in BioKlustering-Website/BioKlustering/mlmodel/parser/spectralClustering.py
### These methods are copy-pasted instead of directly included due to difficulties importing Django classes for running locally without running the server
###

# parseFasta(data) credit to Luke
def parseFasta(data):
    d = {fasta.id : str(fasta.seq) for fasta in SeqIO.parse(data, "fasta")}
    pd.DataFrame([d])
    s = pd.Series(d, name='Sequence')
    s.index.name = 'ID'
    s.reset_index()
    return pd.DataFrame(s)

def get_ids_from_fasta(sequence_paths):
    retval = []
    for path in sequence_paths:
        d = {fasta.id : str(fasta.seq) for fasta in SeqIO.parse(path, "fasta")}
        retval = np.concatenate((np.array(retval), np.array(list(d.keys()))))
    return retval

# this method credit to Zhiwen
def get_kmer_table(paths,k_min,k_max):
    genes,gene_len,output_df = read_fasta(paths)
    count_vect = CountVectorizer(analyzer='char', ngram_range=(k_min, k_max))
    X = count_vect.fit_transform(genes)
    chars = count_vect.get_feature_names_out()
    kmers = X.toarray()
    kmer_freq = []
    for i in range(len(genes)):
        kmer_freq.append(kmers[i] / gene_len[i])
    input = pd.DataFrame(kmer_freq, columns=chars)
    return input, output_df

# this method credit to Zhiwen
def get_gene_sequences(filename):
    genes = []
    for record in SeqIO.parse(filename, "fasta"):
        genes.append(str(record.seq))
    return genes

# this method credit to Zhiwen
# genes: a list of gene sequences, which can directly be generated from get_gene_sequences().
def get_gene_len(genes):
    gene_len = []

    for i in range(len(genes)):
        gene_len.append(len(genes[i]))
    return gene_len

# this method credit to Zhiwen
def read_fasta(paths):
    all_genes = []
    all_gene_len = []
    output_df = pd.DataFrame()
    
    for path in paths:
        virus = parseFasta(path)
        output_df = pd.concat([output_df, virus])
        virus = virus.drop_duplicates(keep="last")
        genes_seq = get_gene_sequences(path)
        gene_len = get_gene_len(genes_seq)
        all_genes = all_genes + genes_seq
        all_gene_len = all_gene_len + gene_len
    return all_genes,all_gene_len,output_df

# this method takes predits the input and make prediction using spectral clustering
# paths: a list of strings. contains file paths
# k_min: int. min of kmer
# k_max: int. max of kmer
# num_cluster: int. number of clusters
# assignLabels: a string. the way to assign label at the final stage of spectral clustering. Can be "kmeans" or "discretize"
def spectral_clustering(num_cluster, assignLabels, paths, k_min = 2, k_max = 3, seed = 0):
    kmer_table, output_df = get_kmer_table(paths, k_min, k_max)
    # if len(kmer_table) < num_cluster:
    #     raise ValueError()
    spectral_clustering = SpectralClustering(n_clusters=num_cluster, assign_labels=assignLabels, random_state=0)
    labels = spectral_clustering.fit_predict(kmer_table)
    output_df.insert(0, "Labels", labels)
    return labels,output_df
# this method takes prints the spectral clustering result by using PCA
# paths: a list of strings. contains file paths
# k_min: int. min of kmer
# k_max: int. max of kmer
# num_cluster: int. number of clusters
# assignLabels: a string. the way to assign label at the final stage of spectral clustering. Can be "kmeans" or "discretize"
def PCA_show_spectural_clustering(num_cluster, assignLabels, paths, filename, k_min=2, k_max=3, seed=0):
    kmer_table,output_df = get_kmer_table(paths, k_min, k_max);
    prediction = SpectralClustering(n_clusters=num_cluster, assign_labels=assignLabels, random_state=seed).fit_predict(
        kmer_table)
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(kmer_table)
    d = {'dimension1': pca_result[:, 0], 'dimension2': pca_result[:, 1], 'label': prediction}
    df = pd.DataFrame(d)
    for i in range(num_cluster):
        label = df.loc[df['label'] == i]
        color = 'C' + str(i)
        plt.scatter(label['dimension1'].tolist(), label['dimension2'].tolist(), c=color)
    plt.xlabel('principal component 1')
    plt.ylabel('principal component 2')
    plt.title('Unsupervised Spectral clustring with ' + assignLabels + ' assign label method')
    df.to_csv('spectral/' + filename + "_PCA.csv")

# NOTE: do we also want to show users the optimal k_min and k_max for users here?
def intuitive_semi_supervised(num_cluster, file_path, label_path, assignLabels = "none", k_min=2, k_max=3, seed=699):
    labels = pd.read_csv(label_path)
    label_list = labels["Labels"].to_list()
    inputlabels = labels["Labels"].to_list()

    IDs = get_ids_from_fasta(file_path)
    unique_given_labels = get_unique_numbers(label_list)
    if num_cluster < len(unique_given_labels) - 1 and -1 in unique_given_labels:
        num_cluster = len(unique_given_labels) - 1
    if num_cluster < len(unique_given_labels) and -1 not in unique_given_labels:
        num_cluster = len(unique_given_labels)
    total_len = len(label_list)
    unknown_label = -1
    total_labeled = 0
    optimal_accuracy = 0
    optimal_k_min = 0
    optimal_k_max = 0
    kmer_table = pd.DataFrame(data={})
    output_df = pd.DataFrame(data={})
    for i in label_list:
        if label_list[i] != unknown_label:
            total_labeled = total_labeled + 1
    res = [0] * total_len
    if assignLabels == "none":
        for i in range(k_min, k_max + 1):
            for j in range(i, k_max + 1):
                temp_k_min = i
                temp_k_max = j
                kmer_table, output_df = get_kmer_table(file_path, temp_k_min, temp_k_max)
                spectral_clustering = SpectralClustering(n_clusters=num_cluster, assign_labels="kmeans",
                                                         random_state=seed)
                labels = spectral_clustering.fit_predict(kmer_table)

                given_labels_count = {}
                labels_list = list(inputlabels)
                for label in unique_given_labels:
                    given_labels_count[label] = labels_list.count(label)
                unique_predicted_labels = get_unique_numbers(labels)
                predicted_labels_count = {}
                for label in unique_predicted_labels:
                    predicted_labels_count[label] = (labels == label).sum()
                max_item = max(predicted_labels_count, key=predicted_labels_count.get)
                if -1 in given_labels_count.keys():
                    del given_labels_count[-1]
                given_labels_count = sorted(given_labels_count.items(), key=lambda x: x[1], reverse=True)
                predicted_labels_count = sorted(predicted_labels_count.items(), key=lambda x: x[1], reverse=True)

                unselected_given = copy.deepcopy(unique_given_labels)
                if -1 in unselected_given:
                    unselected_given.remove(-1)
                unselected_pred = copy.deepcopy(unique_predicted_labels)
                map_predict_to_actual = {}
                for label_GIVEN_dict_entry in given_labels_count:
                    label_GIVEN = label_GIVEN_dict_entry[0]
                    predicted_labels_count_GIVEN = {}
                    label_GIVEN_idx = [index for (index, item) in enumerate(labels_list) if item == label_GIVEN]
                    res_GIVEN = [labels[k] for k in label_GIVEN_idx]
                    unique_predicted_labels_GIVEN = list(set(get_unique_numbers(res_GIVEN)) & set(unselected_pred))
                    if len(unique_predicted_labels_GIVEN) == 0:
                        continue
                    for lab in unique_predicted_labels_GIVEN:
                        predicted_labels_count_GIVEN[lab] = (res_GIVEN == lab).sum()
                    map_predict_to_actual[max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get)] = label_GIVEN
                    unselected_given.remove(label_GIVEN)
                    unselected_pred.remove(max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get))
                    
                # in the case where multiple given labels completely map to the same 
                # predicted label, we need to finish assigning given labels to any 
                # predicted label
                for lab_remain in unselected_given:
                    for upl in unique_predicted_labels:
                        if upl not in map_predict_to_actual.keys():
                            map_predict_to_actual[upl] = lab_remain
                            unselected_given.remove(lab_remain)
                            unselected_pred.remove(upl)
                            break


                if len(unique_given_labels) <= num_cluster:
                    max_value = max(unique_given_labels) + 1
                    for upl in unique_predicted_labels:
                        if upl not in map_predict_to_actual.keys():
                            map_predict_to_actual[upl] = max_value
                            max_value += 1
                            unselected_pred.remove(upl)
                
                for l in range(len(unselected_given)):
                    map_predict_to_actual[unselected_pred[l]] = unselected_given[l]
                    

                # predictions_final contains the final results
                # it takes care of the case when num_class > number of unique labels given
                predictions_tmp = []
                for k in range(len(labels)):
                    if labels[k] in map_predict_to_actual.keys():
                        predictions_tmp.append(map_predict_to_actual[labels[k]])
                    else:
                        predictions_tmp.append(map_predict_to_actual[max_item])

                correct_count = 0
                for k in range(len(label_list)):
                    if label_list[k] != unknown_label:
                        if label_list[k] == predictions_tmp[k]:
                            correct_count += 1
                temp_accuracy = correct_count / total_labeled
                if temp_accuracy > optimal_accuracy:
                    optimal_accuracy = temp_accuracy
                    optimal_k_min = i
                    optimal_k_max = j
                    res = labels
        for i in range(k_min, k_max + 1):
            for j in range(i, k_max + 1):
                temp_k_min = i
                temp_k_max = j
                kmer_table, output_df = get_kmer_table(file_path, temp_k_min, temp_k_max)
                spectral_clustering = SpectralClustering(n_clusters=num_cluster, assign_labels="discretize",
                                                         random_state=seed)
                labels = spectral_clustering.fit_predict(kmer_table)

                given_labels_count = {}
                labels_list = list(inputlabels)
                for label in unique_given_labels:
                    given_labels_count[label] = labels_list.count(label)
                unique_predicted_labels = get_unique_numbers(labels)
                predicted_labels_count = {}
                for label in unique_predicted_labels:
                    predicted_labels_count[label] = (labels == label).sum()
                max_item = max(predicted_labels_count, key=predicted_labels_count.get)
                if -1 in given_labels_count.keys():
                    del given_labels_count[-1]
                given_labels_count = sorted(given_labels_count.items(), key=lambda x: x[1], reverse=True)
                predicted_labels_count = sorted(predicted_labels_count.items(), key=lambda x: x[1], reverse=True)

                unselected_given = copy.deepcopy(unique_given_labels)
                if -1 in unselected_given:
                    unselected_given.remove(-1)
                unselected_pred = copy.deepcopy(unique_predicted_labels)

                map_predict_to_actual = {}
                for label_GIVEN_dict_entry in given_labels_count:
                    label_GIVEN = label_GIVEN_dict_entry[0]
                    predicted_labels_count_GIVEN = {}
                    label_GIVEN_idx = [index for (index, item) in enumerate(labels_list) if item == label_GIVEN]
                    res_GIVEN = [labels[k] for k in label_GIVEN_idx]
                    unique_predicted_labels_GIVEN = list(set(get_unique_numbers(res_GIVEN)) & set(unselected_pred))
                    if len(unique_predicted_labels_GIVEN) == 0:
                        continue
                    for lab in unique_predicted_labels_GIVEN:
                        predicted_labels_count_GIVEN[lab] = (res_GIVEN == lab).sum()
                    map_predict_to_actual[max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get)] = label_GIVEN
                    unselected_given.remove(label_GIVEN)
                    unselected_pred.remove(max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get))


                # in the case where multiple given labels completely map to the same 
                # predicted label, we need to finish assigning given labels to any 
                # predicted label
                for lab_remain in unselected_given:
                    for upl in unique_predicted_labels:
                        if upl not in map_predict_to_actual.keys():
                            map_predict_to_actual[upl] = lab_remain
                            unselected_given.remove(lab_remain)
                            unselected_pred.remove(upl)
                            break
                        
                if len(unique_given_labels) <= num_cluster:
                    max_value = max(unique_given_labels) + 1
                    for upl in unique_predicted_labels:
                        if upl not in map_predict_to_actual.keys():
                            map_predict_to_actual[upl] = max_value
                            max_value += 1
                            unselected_pred.remove(upl)
                
                for l in range(len(unselected_given)):
                    map_predict_to_actual[unselected_pred[l]] = unselected_given[l]

                # predictions_final contains the final results
                # it takes care of the case when num_class > number of unique labels given
                predictions_tmp = []
                for k in range(len(labels)):
                    if labels[k] in map_predict_to_actual.keys():
                        predictions_tmp.append(map_predict_to_actual[labels[k]])
                    else:
                        predictions_tmp.append(map_predict_to_actual[max_item])

                correct_count = 0
                for k in range(len(label_list)):
                    if label_list[k] != unknown_label:
                        if label_list[k] == predictions_tmp[k]:
                            correct_count += 1
                temp_accuracy = correct_count / total_labeled
                if temp_accuracy > optimal_accuracy:
                    optimal_accuracy = temp_accuracy
                    optimal_k_min = i
                    optimal_k_max = j
                    res = labels

    else:
        for i in range(k_min, k_max + 1):
            for j in range(i, k_max + 1):
                temp_k_min = i
                temp_k_max = j
                kmer_table, output_df = get_kmer_table(file_path, temp_k_min, temp_k_max)
                spectral_clustering = SpectralClustering(n_clusters=num_cluster, assign_labels=assignLabels,
                                                         random_state=seed)
                labels = spectral_clustering.fit_predict(kmer_table)

                # Get the counts for the given labels and the predicted labels
                given_labels_count = {}
                labels_list = list(inputlabels)
                for label in unique_given_labels:
                    given_labels_count[label] = labels_list.count(label)
                unique_predicted_labels = get_unique_numbers(labels)
                predicted_labels_count = {}
                for label in unique_predicted_labels:
                    predicted_labels_count[label] = (labels == label).sum()
                max_item = max(predicted_labels_count, key=predicted_labels_count.get)
                if -1 in given_labels_count.keys():
                    del given_labels_count[-1]
                given_labels_count = sorted(given_labels_count.items(), key=lambda x: x[1], reverse=True)
                predicted_labels_count = sorted(predicted_labels_count.items(), key=lambda x: x[1], reverse=True)

                # Map the predicted labels to the given/actual labels
                unselected_given = copy.deepcopy(unique_given_labels)
                if -1 in unselected_given:
                    unselected_given.remove(-1)
                unselected_pred = copy.deepcopy(unique_predicted_labels)
                
                map_predict_to_actual = {}
                for label_GIVEN_dict_entry in given_labels_count:
                    label_GIVEN = label_GIVEN_dict_entry[0]
                    predicted_labels_count_GIVEN = {}
                    label_GIVEN_idx = [index for (index, item) in enumerate(labels_list) if item == label_GIVEN]
                    res_GIVEN = [labels[k] for k in label_GIVEN_idx]
                    unique_predicted_labels_GIVEN = list(set(get_unique_numbers(res_GIVEN)) & set(unselected_pred))
                    if len(unique_predicted_labels_GIVEN) == 0:
                        continue
                    for lab in unique_predicted_labels_GIVEN:
                        predicted_labels_count_GIVEN[lab] = (res_GIVEN == lab).sum()
                    map_predict_to_actual[max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get)] = label_GIVEN
                    unselected_given.remove(label_GIVEN)
                    unselected_pred.remove(max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get))

                
                # in the case where multiple given labels completely map to the same 
                # predicted label, we need to finish assigning given labels to any 
                # predicted label
                for lab_remain in unselected_given:
                    for upl in unique_predicted_labels:
                        if upl not in map_predict_to_actual.keys():
                            map_predict_to_actual[upl] = lab_remain
                            unselected_given.remove(lab_remain)
                            unselected_pred.remove(upl)
                            break
                                        

                if len(unique_given_labels) <= num_cluster:
                    max_value = max(unique_given_labels) + 1
                    for upl in unique_predicted_labels:
                        if upl not in map_predict_to_actual.keys():
                            map_predict_to_actual[upl] = max_value
                            max_value += 1
                            unselected_pred.remove(upl)
                            
                if len(unselected_given) != len(unselected_pred):
                    print("error: num unselected given =",len(unselected_given), "!= unselected pred =",len(unselected_pred))
                    
                print(f"map_predict_to_actual: {map_predict_to_actual}")
            
                for l in range(len(unselected_given)):
                    map_predict_to_actual[unselected_pred[l]] = unselected_given[l]

                # predictions_final contains the final results
                # it takes care of the case when num_class > number of unique labels given
                predictions_tmp = []
                for k in range(len(labels)):
                    if labels[k] in map_predict_to_actual.keys():
                        predictions_tmp.append(map_predict_to_actual[labels[k]])
                    else:
                        predictions_tmp.append(map_predict_to_actual[max_item])

                correct_count = 0
                temp_accuracy = 0
                for k in range(len(label_list)):
                    if label_list[k] != unknown_label:
                        if label_list[k] == predictions_tmp[k]:
                            correct_count += 1
                temp_accuracy = correct_count / total_labeled
                if temp_accuracy > optimal_accuracy:
                    optimal_accuracy = temp_accuracy
                    optimal_k_min = i
                    optimal_k_max = j
                    res = labels

    res = np.array(res)

    # Get the counts for the given labels and the predicted labels
    given_labels_count = {}
    labels_list = list(inputlabels)
    for label in unique_given_labels:
        given_labels_count[label] = labels_list.count(label)
    unique_predicted_labels = get_unique_numbers(res)
    predicted_labels_count = {}
    for label in unique_predicted_labels:
        predicted_labels_count[label] = (res == label).sum()
    max_item = max(predicted_labels_count, key=predicted_labels_count.get)
    if -1 in given_labels_count.keys():
        del given_labels_count[-1]
    given_labels_count = sorted(given_labels_count.items(), key=lambda x: x[1], reverse=True)
    predicted_labels_count = sorted(predicted_labels_count.items(), key=lambda x: x[1], reverse=True)

    # Map the predicted labels to the given/actual labels
    unselected_given = copy.deepcopy(unique_given_labels)
    if -1 in unselected_given:
        unselected_given.remove(-1)
    unselected_pred = copy.deepcopy(unique_predicted_labels)

    map_predict_to_actual = {}
    for label_GIVEN_dict_entry in given_labels_count:
        label_GIVEN = label_GIVEN_dict_entry[0]
        predicted_labels_count_GIVEN = {}
        label_GIVEN_idx = [index for (index, item) in enumerate(labels_list) if item == label_GIVEN]
        res_GIVEN = [res[k] for k in label_GIVEN_idx]
        unique_predicted_labels_GIVEN = list(set(get_unique_numbers(res_GIVEN)) & set(unselected_pred))
        if len(unique_predicted_labels_GIVEN) == 0:
            continue
        for lab in unique_predicted_labels_GIVEN:
            predicted_labels_count_GIVEN[lab] = (res_GIVEN == lab).sum()
        map_predict_to_actual[max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get)] = label_GIVEN
        unselected_given.remove(label_GIVEN)
        unselected_pred.remove(max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get))


    # in the case where multiple given labels completely map to the same 
    # predicted label, we need to finish assigning given labels to any 
    # predicted label
    for lab_remain in unselected_given:
        for upl in unique_predicted_labels:
            if upl not in map_predict_to_actual.keys():
                map_predict_to_actual[upl] = lab_remain
                unselected_given.remove(lab_remain)
                unselected_pred.remove(upl)
                break
            
    if len(unique_given_labels) <= num_cluster:
        max_value = max(unique_given_labels) + 1
        for upl in unique_predicted_labels:
            if upl not in map_predict_to_actual.keys():
                # print(f"{upl} mapped to {max_value}")
                map_predict_to_actual[upl] = max_value
                max_value += 1
                unselected_pred.remove(upl)
    
    if len(unselected_given) != len(unselected_pred):
        print("error: num unselected given =",len(unselected_given), "!= unselected pred =",len(unselected_pred))
        
    print(f"map_predict_to_actual: {map_predict_to_actual}")
    
    for l in range(len(unselected_given)):
        map_predict_to_actual[unselected_pred[l]] = unselected_given[l]


    # predictions_final contains the final results
    # it takes care of the case when num_class > number of unique labels given
    predictions_final = []
    predictions_tmp = []
    for i in range(len(res)):
        if inputlabels[i] == -1:
            if res[i] in map_predict_to_actual.keys():
                predictions_final.append(map_predict_to_actual[res[i]])
            else:
                predictions_final.append(map_predict_to_actual[max_item])
        else:
            predictions_final.append(inputlabels[i])
        if res[i] in map_predict_to_actual.keys():
            predictions_tmp.append(map_predict_to_actual[res[i]])
        else:
            predictions_tmp.append(map_predict_to_actual[max_item])
    res = np.array(predictions_final) 
    
    return res, np.array(predictions_tmp)
    
def PCA_show_semi_spectural_clustering(num_cluster, file_path, label_path, filename, assignLabels = "none", k_min=2, k_max=3, seed=699):
    prediction,_ = intuitive_semi_supervised(num_cluster,  file_path, label_path, assignLabels, k_min, k_max, seed)
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(kmer_table)
    d = {'dimension1': pca_result[:, 0], 'dimension2': pca_result[:, 1], 'label': prediction}
    df = pd.DataFrame(d)
    for i in range(num_cluster):
        label = df.loc[df['label'] == i]
        color = 'C' + str(i)
        plt.scatter(label['dimension1'].tolist(), label['dimension2'].tolist(), c=color)
    plt.xlabel('principal component 1')
    plt.ylabel('principal component 2')
    plt.title('Semi-supervised Spectral clustring with ' + assignLabels + ' assign label method')
    df.to_csv('spectral/' + filename + "_PCA.csv")

def get_unique_numbers(numbers):

    list_of_unique_numbers = []

    unique_numbers = set(numbers)

    for number in unique_numbers:
        list_of_unique_numbers.append(number)

    return list_of_unique_numbers

# Nucleotide

In [40]:
path = ["../combined_nucleotide.fasta"]
lab_path = "../combined_labels_nucleotide.csv"
k_min = 2
k_max = 3
num_classes = 2
seed = 32624
predictions_nuc, tmp = intuitive_semi_supervised(num_classes,path,lab_path,"none",k_min,k_max,seed)

kmer_table,output_df = get_kmer_table(path, k_min, k_max)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(kmer_table)
d = {'ID': pd.Series(output_df.index), 'principal component 1': pca_result[:, 0], 'principal component 2': pca_result[:, 1], 'label': predictions_nuc}
finalDf = pd.DataFrame(d)
finalDf.columns = ['ID','principal Component 1', 'principal Component 2','label']
finalDf.to_csv('../results/nuc_spectral_pca.csv',index = False)

map_predict_to_actual: {1: 0, 0: 1}


In [41]:
labels_nuc = pd.Series(pd.read_csv("../combined_labels_nucleotide.csv").Labels)
n_unlab = sum(labels_nuc == -1)

print("Ones: ", sum(predictions_nuc), " out of ", n_unlab)
print("Zeros: ", sum(1- predictions_nuc), " out of ", n_unlab)

Ones:  207  out of  350
Zeros:  159  out of  350


# Amino Acids

In [42]:
path = ["../combined_amino.fasta"]
lab_path = "../combined_labels_amino.csv"
k_min = 4
k_max = 5
num_classes = 2
seed = 32624
predictions_am, tmp = intuitive_semi_supervised(num_classes,path,lab_path,"none",k_min,k_max,seed)

kmer_table,output_df = get_kmer_table(path, k_min, k_max)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(kmer_table)
d = {'ID': pd.Series(output_df.index), 'principal component 1': pca_result[:, 0], 'principal component 2': pca_result[:, 1], 'label': predictions_am}
finalDf = pd.DataFrame(d)
finalDf.columns = ['ID','principal Component 1', 'principal Component 2','label']
finalDf.to_csv('../results/am_spectral_pca.csv',index = False)

map_predict_to_actual: {0: 0, 1: 1}


In [43]:
labels_am = pd.Series(pd.read_csv("../combined_labels_amino.csv").Labels)
n_unlab = sum(labels_am == -1)

print("Ones: ", sum(predictions_am), " out of ", n_unlab)
print("Zeros: ", sum(1- predictions_am), " out of ", n_unlab)

Ones:  8  out of  465
Zeros:  473  out of  465
